In [92]:
import csv
import json

from merkle import *

In [58]:
with open('data/ownership.csv', 'r') as f:
    reader = csv.reader(f)
    ownership_tuples= list(reader)
    
index_tuples = []
i = 0
for t in ownership_tuples:
    index_tuples.append([i, t[0], t[1], int(t[2])])
    i += 1

Ok, let's merkle tree up this list of leaves with keccak256 goodness

Create a map of merkle leaf -> branch and store the root. Then serialize this entire object in json.

Ah-ha! There's a way to do this in web3py: https://web3py.readthedocs.io/en/stable/web3.main.html?highlight=solidityKeccak#web3.Web3.solidityKeccak

Merkle alg:
- store leaf -> [branch]
- store last_layer_node -> leaf
- until len(last_layer) == 1
  - hash pairs of last_layer to create new layer
  - add other side of pair + new parent to branches[leaf[last_layer_node]]
- return size 1 last_layer (root) + leaf -> [branch]

In [93]:
tree = build_tree(index_tuples)

Ok, now we should keep these somewhere handy. Next step is to *verify* each merkle branch. If any don't verify, note that somewhere.

Let's just make sure they're all consistent as a final sanity check on today's work.

Well.... shit. They don't match. It's the ordering thing! I need to have a standard ordering for the combination method. And this should be do-able within the contract as well!

Sure enough, there's a openzepplin lib I can use for this and that lib sorts before combining: https://github.com/OpenZeppelin/openzeppelin-contracts/blob/master/contracts/cryptography/MerkleProof.sol#L15-L32

In [81]:
leaf_to_branch = tree.leaf_to_branch
root = tree.root

In [87]:
for leaf, branch in leaf_to_branch.items():
    if not check_branch(root, leaf, branch):
        print("roots don't match!")

Success! Let's store the merkle data in json and prep it for sharing later.

In [95]:
with open('data/merkle_tree.json', 'w') as f:
    json.dump(tree.jsonify(), f)